# HIMALAYAS

### In brief...

### For every iteration, the df is inspected and then filtered by its pubDate (if there's one). Finally, it creates a new df that will be transformed into a new table in the local postgreSQL DB.

In [5]:
import pandas as pd
import matplotlib.pyplot as plt 
import mysql.connector
import psycopg2
import numpy as np
from dateutil.relativedelta import relativedelta
from dateutil.parser import parse

In [6]:
df = pd.read_csv('/Users/juanreyesgarcia/Library/CloudStorage/OneDrive-FundacionUniversidaddelasAmericasPuebla/DEVELOPER/PROJECTS/CRAWLER_ALL/OUTPUTS/himalaya.csv')

##### Play with the settings...

In [7]:
pd.set_option('display.max_colwidth', 150)

In [8]:
pd.set_option("display.max_rows", None)

# EDA Checklist 

In [9]:
df[:100]

,title,link,pubDate,location,description
0,"['Product Manager, Financial Crimes']",https://himalayas.app/companies/affirm/jobs/product-manager-financial-crimes,1 hour ago,Poland only,Product Manager
1,['Information Security Specialist'],https://himalayas.app/companies/monzo/jobs/information-security-specialist-8019255294,3 hours ago,United Kingdom only,Information Security Developer
2,['Accounting Assistant'],https://himalayas.app/companies/lullabot/jobs/accounting-assistant,4 hours ago,United States only,Accountant
3,['Staff DevOps Engineer'],https://himalayas.app/companies/postscript/jobs/staff-devops-engineer,5 hours ago,North America only,DevOps Engineer
4,['Head of Account Management'],https://himalayas.app/companies/mercury/jobs/head-of-account-management,5 hours ago,US &amp; CA,Account Manager
5,['Senior Software Engineer - Backend'],https://himalayas.app/companies/ipinfo/jobs/senior-software-engineer-backend,5 hours ago,US &amp; CA,Back End Developer
6,['Relationship Management Intern'],https://himalayas.app/companies/mercury/jobs/relationship-management-intern,5 hours ago,Poland only,Relationship Manager
7,['Senior Infrastructure Engineer'],https://himalayas.app/companies/prisma/jobs/senior-infrastructure-engineer,5 hours ago,United States only,Infrastructure Engineer
8,"['Staff Software Engineer, Mobile (Debit+)']",https://himalayas.app/companies/affirm/jobs/staff-software-engineer-mobile-debit,7 hours ago,Singapore only,Software Engineer
9,['Customer Support Representative'],https://himalayas.app/companies/roofr/jobs/customer-support-representative-832726288,8 hours ago,United States only,Customer Support Representative


In [6]:
df.tail()

,title,link,pubDate,location,description
55,['Senior Accountant'],https://himalayas.app/companies/affirm/jobs/senior-accountant,1 day ago,NaN,Accountant
56,"['Associate, Strategic Projects']",https://himalayas.app/companies/rhino/jobs/associate-strategic-projects,1 day ago,NaN,Project Manager
57,['Field Marketing Manager'],https://himalayas.app/companies/kojo/jobs/field-marketing-manager,1 day ago,NaN,Marketing Manager
58,"['Sr. Product Manager, Virtual Vehicle Infotainment &amp; HMI']",https://himalayas.app/companies/platform-science/jobs/sr-product-manager-virtual-vehicle-infotainment-hmi,1 day ago,NaN,Product Manager
59,['CX Workforce Management Lead'],https://himalayas.app/companies/headway-co/jobs/cx-workforce-management-lead,1 day ago,NaN,Head of Human Resources


### Describe the DF (current)

In [7]:
df.describe()

,title,link,pubDate,location,description
count,60,60,60,50,60
unique,58,60,16,14,41
top,"['Marketing Operations Manager, Analytics']",https://himalayas.app/companies/affirm/jobs/product-manager-financial-crimes,1 day ago,United States only,Software Engineer
freq,2,1,31,29,8


### Checking for missing values DF (current)

In [8]:
df.isnull().sum()

title           0
link            0
pubDate         0
location       10
description     0
dtype: int64

### Fill missing values with "NaN" DF (current)

In [9]:
# Fill missing values with "NaN"
df.fillna("NaN", inplace=True)

In [10]:
df.isnull().sum()

title          0
link           0
pubDate        0
location       0
description    0
dtype: int64

### Checking for duplicates DF (current)

In [11]:
df.duplicated().sum()

0

In [12]:
df.describe()

,title,link,pubDate,location,description
count,60,60,60,60,60
unique,58,60,16,15,41
top,"['Marketing Operations Manager, Analytics']",https://himalayas.app/companies/affirm/jobs/product-manager-financial-crimes,1 day ago,United States only,Software Engineer
freq,2,1,31,29,8


In [13]:
df.dtypes

title          object
link           object
pubDate        object
location       object
description    object
dtype: object

### Mask to convert %minutes|hours to 1 day ago so it can be parse to date time (cba to say it was posted today, maybe later should be fixed)

In [14]:
mask = df['pubdate'].str.contains('minutes|hours')

In [15]:
df.loc[mask, 'pubdate'] = "1 day ago"

In [16]:
df.head()

,title,link,pubDate,location,description
0,"['Product Manager, Financial Crimes']",https://himalayas.app/companies/affirm/jobs/product-manager-financial-crimes,1 hour ago,Poland only,Product Manager
1,['Information Security Specialist'],https://himalayas.app/companies/monzo/jobs/information-security-specialist-8019255294,1 day ago,United Kingdom only,Information Security Developer
2,['Accounting Assistant'],https://himalayas.app/companies/lullabot/jobs/accounting-assistant,1 day ago,United States only,Accountant
3,['Staff DevOps Engineer'],https://himalayas.app/companies/postscript/jobs/staff-devops-engineer,1 day ago,North America only,DevOps Engineer
4,['Head of Account Management'],https://himalayas.app/companies/mercury/jobs/head-of-account-management,1 day ago,US &amp; CA,Account Manager


# From relative date strings to date time...

In [17]:
# create a mask to identify rows with relative date strings
mask = df['pubdate'].str.contains('ago')

# apply the relative date calculation to the date_string column
df.loc[mask, 'datetime'] = pd.Timestamp.today() - df.loc[mask, 'pubdate'].apply(lambda x: relativedelta(days=int(x.split()[0])))

# convert the datetime column to datetime format
df['datetime'] = pd.to_datetime(df['datetime'], infer_datetime_format=True)


/var/folders/3x/_059946j4lvcjpvshx_dj5f40000gn/T/ipykernel_19211/1559547877.py:5: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[mask, 'datetime'] = pd.Timestamp.today() - df.loc[mask, 'pubDate'].apply(lambda x: relativedelta(days=int(x.split()[0])))


In [18]:
df.head(50)

,title,link,pubDate,location,description,datetime
0,"['Product Manager, Financial Crimes']",https://himalayas.app/companies/affirm/jobs/product-manager-financial-crimes,1 hour ago,Poland only,Product Manager,2023-03-06 00:16:37.121159
1,['Information Security Specialist'],https://himalayas.app/companies/monzo/jobs/information-security-specialist-8019255294,1 day ago,United Kingdom only,Information Security Developer,2023-03-06 00:16:37.121159
2,['Accounting Assistant'],https://himalayas.app/companies/lullabot/jobs/accounting-assistant,1 day ago,United States only,Accountant,2023-03-06 00:16:37.121159
3,['Staff DevOps Engineer'],https://himalayas.app/companies/postscript/jobs/staff-devops-engineer,1 day ago,North America only,DevOps Engineer,2023-03-06 00:16:37.121159
4,['Head of Account Management'],https://himalayas.app/companies/mercury/jobs/head-of-account-management,1 day ago,US &amp; CA,Account Manager,2023-03-06 00:16:37.121159
5,['Senior Software Engineer - Backend'],https://himalayas.app/companies/ipinfo/jobs/senior-software-engineer-backend,1 day ago,US &amp; CA,Back End Developer,2023-03-06 00:16:37.121159
6,['Relationship Management Intern'],https://himalayas.app/companies/mercury/jobs/relationship-management-intern,1 day ago,Poland only,Relationship Manager,2023-03-06 00:16:37.121159
7,['Senior Infrastructure Engineer'],https://himalayas.app/companies/prisma/jobs/senior-infrastructure-engineer,1 day ago,United States only,Infrastructure Engineer,2023-03-06 00:16:37.121159
8,"['Staff Software Engineer, Mobile (Debit+)']",https://himalayas.app/companies/affirm/jobs/staff-software-engineer-mobile-debit,1 day ago,Singapore only,Software Engineer,2023-03-06 00:16:37.121159
9,['Customer Support Representative'],https://himalayas.app/companies/roofr/jobs/customer-support-representative-832726288,1 day ago,United States only,Customer Support Representative,2023-03-06 00:16:37.121159


### Drop pubDate, change name of datetime & reindex...

In [19]:
#Drop it
df = df.drop('pubdate', axis=1)

In [20]:
#Change name of datetime to pubDate 
df = df.rename(columns={'datetime': 'pubdate'})

In [21]:
#reindex
df = df.reindex(columns=['title', 'link', 'description', 'pubdate', 'location'])

In [22]:
df.head()

,title,link,description,pubDate,location
0,"['Product Manager, Financial Crimes']",https://himalayas.app/companies/affirm/jobs/product-manager-financial-crimes,Product Manager,2023-03-06 00:16:37.121159,Poland only
1,['Information Security Specialist'],https://himalayas.app/companies/monzo/jobs/information-security-specialist-8019255294,Information Security Developer,2023-03-06 00:16:37.121159,United Kingdom only
2,['Accounting Assistant'],https://himalayas.app/companies/lullabot/jobs/accounting-assistant,Accountant,2023-03-06 00:16:37.121159,United States only
3,['Staff DevOps Engineer'],https://himalayas.app/companies/postscript/jobs/staff-devops-engineer,DevOps Engineer,2023-03-06 00:16:37.121159,North America only
4,['Head of Account Management'],https://himalayas.app/companies/mercury/jobs/head-of-account-management,Account Manager,2023-03-06 00:16:37.121159,US &amp; CA


## Filter rows by a date range (this reduces the number of rows... duh)

In [23]:
# filter rows by a date range
start_date = pd.to_datetime('2016-01-01')
end_date = pd.to_datetime('2023-02-15')

#for df
date_range_filter = (df['pubDate'] >= start_date) & (df['pubDate'] <= end_date)
df = df.loc[~date_range_filter]

In [24]:
df = df.sort_values(by='pubDate')

In [25]:
df.describe(datetime_is_numeric=True)

,pubDate
count,60
mean,2023-03-06 00:16:37.121159168
min,2023-03-06 00:16:37.121159
25%,2023-03-06 00:16:37.121158912
50%,2023-03-06 00:16:37.121158912
75%,2023-03-06 00:16:37.121158912
max,2023-03-06 00:16:37.121159


In [26]:
df.describe()

/var/folders/3x/_059946j4lvcjpvshx_dj5f40000gn/T/ipykernel_19211/3627053830.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df.describe()


,title,link,description,pubDate,location
count,60,60,60,60,60
unique,58,60,41,1,15
top,['Sales Development Representative'],https://himalayas.app/companies/affirm/jobs/product-manager-financial-crimes,Software Engineer,2023-03-06 00:16:37.121159,United States only
freq,2,1,8,60,29
first,NaN,NaN,NaN,2023-03-06 00:16:37.121159,NaN
last,NaN,NaN,NaN,2023-03-06 00:16:37.121159,NaN


In [27]:
df.dtypes

title                  object
link                   object
description            object
pubDate        datetime64[ns]
location               object
dtype: object

### Make a copy just in case

In [28]:
df1 = df.copy()

In [29]:
# replace NaT values in the DataFrame with None
df = df.replace({np.nan: None, pd.NaT: None})

In [30]:
df

,title,link,description,pubDate,location
0,"['Product Manager, Financial Crimes']",https://himalayas.app/companies/affirm/jobs/product-manager-financial-crimes,Product Manager,2023-03-06 00:16:37.121159,Poland only
32,['Sr. Marketing Operations Manager'],https://himalayas.app/companies/classdojo/jobs/sr-marketing-operations-manager,Marketing Operations,2023-03-06 00:16:37.121159,United States only
33,['Senior Software Engineer - Front End'],https://himalayas.app/companies/degreed/jobs/senior-software-engineer-front-end,Software Engineer,2023-03-06 00:16:37.121159,United States only
34,"['Senior Manager, Community Support (m/f/d)']",https://himalayas.app/companies/pitch/jobs/senior-manager-community-support-m-f-d,Community Manager,2023-03-06 00:16:37.121159,Canada only
35,"['Mgr., Support Engineering, Marketplace Solutions']",https://himalayas.app/companies/platform-science/jobs/mgr-support-engineering-marketplace-solutions,Support Engineer,2023-03-06 00:16:37.121159,United States only
36,['Growth Marketing Manager'],https://himalayas.app/companies/grow-therapy/jobs/growth-marketing-manager,Marketing Manager,2023-03-06 00:16:37.121159,United States only
37,['Technical Support Engineer (German Speaking) - EMEA Remote'],https://himalayas.app/companies/degreed/jobs/technical-support-engineer-german-speaking-emea-remote,Technical Support Engineer,2023-03-06 00:16:37.121159,CA &amp; US
38,['Staff Security Engineer'],https://himalayas.app/companies/grow-therapy/jobs/staff-security-engineer,Security Engineer,2023-03-06 00:16:37.121159,United States only
39,['Senior NetSuite Developer'],https://himalayas.app/companies/pleo/jobs/senior-netsuite-developer-6658170280,Software Engineer,2023-03-06 00:16:37.121159,Poland only
40,['Revenue Accounting Manager'],https://himalayas.app/companies/platform-science/jobs/revenue-accounting-manager-7787104513,Accounting Manager,2023-03-06 00:16:37.121159,United States only


# PostgreSQL

### This code creates a new table per iteration

In [31]:
# create a connection to the PostgreSQL database
cnx = psycopg2.connect(user='postgres', password='3312', host='localhost', port='5432', database='postgres')

# create a cursor object
cursor = cnx.cursor()

# get the name of the next table to create
get_table_name_query = '''
    SELECT COUNT(*) FROM information_schema.tables
    WHERE table_name LIKE 'himalayas_%'
'''
cursor.execute(get_table_name_query)
result = cursor.fetchone()
next_table_number = result[0] + 1
next_table_name = 'himalayas_{}'.format(next_table_number)

# prepare the SQL query to create a new table
create_table_query = '''
    CREATE TABLE {} (
        title VARCHAR(255),
        link VARCHAR(255),
        description VARCHAR(1000),
        pubdate TIMESTAMP,
        location VARCHAR(255)
    )
'''.format(next_table_name)

# execute the create table query
cursor.execute(create_table_query)

# insert the DataFrame into the PostgreSQL database as a new table
for index, row in df.iterrows():
    insert_query = '''
        INSERT INTO {} (title, link, description, pubdate, location)
        VALUES (%s, %s, %s, %s, %s)
    '''.format(next_table_name)
    values = (row['title'], row['link'], row['description'], row['pubdate'], row['location'])
    cursor.execute(insert_query, values)

# commit the changes to the database
cnx.commit()

# close the cursor and connection
cursor.close()
cnx.close()